### 4th European Conference On Social Networks
## Workshop: Introduction to Python's Graph-Tool
# 2. Bipartite Graphs
**Authors**: <a href='https://www.gesis.org/person/haiko.lietz'>Haiko Lietz</a>, <a href='https://www.gesis.org/person/marcos.oliveira'>Marcos Oliveira</a>, GESIS - Leibniz Institute for the Social Sciences

**Version Date**: 12 September 2019

**Description**: This notebook introduces ...

**License**: <a href='https://www.gnu.org/licenses/gpl-3.0.en.html'>GNU General Public License 3.0</a>
***
<img src = 'images/bipartite.png'>

## 2.1. Toy Example

In [ ]:
import pandas as pd

In [ ]:
selections = [['a_1', 'f_1'], ['a_1', 'f_2'], ['a_1', 'f_3'], ['a_1', 'f_4'], ['a_1', 'f_5'], ['a_2', 'f_3'], ['a_2', 'f_4'], ['a_2', 'f_5'], ['a_2', 'f_6'], ['a_3', 'f_5'], ['a_3', 'f_6'], ['a_3', 'f_7'], ['a_4', 'f_7'], ['a_4', 'f_8'], ['a_5', 'f_9']]
selections = pd.DataFrame(selections, columns=['transaction', 'fact'])
selections['weight'] = 1
selections.head()

### 2.1.1. Matrix Projection

In [ ]:
def project_bipartite(df_selections, projection, norm=True, remove_diagonal=False, sym=False):
    # dependencies
    import itertools
    import numpy as np
    import pandas as pd
    from scipy.sparse import csr_matrix, coo_matrix, triu
    from sklearn.preprocessing import normalize
    # function
    if {'transaction', 'fact', 'weight'}.issubset(df_selections.columns):
        def extract_vertices(df, *columns):
            l = [df[column].unique().tolist() for column in columns]
            return list(set(itertools.chain.from_iterable(l)))
        transactions = extract_vertices(df_selections, 'transaction')
        transactions_id = {value: i for i, value in enumerate(transactions)}
        facts = extract_vertices(df_selections, 'fact')
        facts_id = {value: i for i, value in enumerate(facts)}
        rows = [transactions_id[x] for x in df_selections['transaction'].values]
        columns = [facts_id[y] for y in df_selections['fact'].values]
        cells = df_selections['weight'].tolist()
        G = coo_matrix((cells, (rows, columns))).tocsr()
        if projection == 'transactions':
            if norm == False:
                GT = csr_matrix.transpose(G)
                I = G*GT
            else:
                GN = normalize(G, norm='l1', axis=1)
                GNT = csr_matrix.transpose(GN)
                I = GN*GNT
            transactions = pd.DataFrame(transactions)
            transactions.columns = ['transaction']
            if remove_diagonal == True:
                I = I.tolil()
                I.setdiag(0)
            if sym == False:
                return I.tocsr(), transactions
            else:
                return triu(I.tocoo()).tocsr(), transactions
        if projection == 'facts':
            GT = csr_matrix.transpose(G)
            if norm == False:
                H = GT*G
            else:
                GN = normalize(G, norm='l1', axis=1)
                H = GT*GN
            w = pd.Series(np.squeeze(np.array(H.sum(axis=1))))
            d = pd.Series(np.array(H.diagonal()))
            e = d/w
            H_nodiag = H.tolil()
            H_nodiag.setdiag(values=0)
            k = pd.Series(np.array([len(i) for i in H_nodiag.data.tolist()]))
            s = k/w
            facts = pd.Series(facts)
            facts = pd.concat([facts, k, w, d, e, s], axis=1)
            facts.columns = ['fact', 'degree', 'weight', 'self_selection', 'embeddedness', 'sociability']
            if remove_diagonal == True:
                H = H.tolil()
                H.setdiag(0)
            if sym == False:
                return H.tocsr(), facts
            else:
                return triu(H.tocoo()).tocsr(), facts
    else:
        print('Dataframe is not a proper selection table.')

In [ ]:
I, transactions = project_bipartite(df_selections=selections, projection='transactions', norm=False, remove_diagonal=True, sym=True)

In [ ]:
I_norm, _ = project_bipartite(df_selections=selections, projection='transactions', norm=True, remove_diagonal=True, sym=True)

In [ ]:
transactions

In [ ]:
print(I)

In [ ]:
print(I_norm)

In [ ]:
H, facts = project_bipartite(df_selections=selections, projection='facts', norm=False, remove_diagonal=True, sym=True)

In [ ]:
H_norm, facts_norm = project_bipartite(df_selections=selections, projection='facts', norm=True, remove_diagonal=True, sym=True)

In [ ]:
facts

In [ ]:
facts_norm

In [ ]:
print(H)

In [ ]:
print(H_norm)

### 2.1.2. Graph Construction

In [ ]:
i = H_norm.nonzero()[0].tolist()
j = H_norm.nonzero()[1].tolist()
weight = H_norm.data.tolist()

In [ ]:
i

In [ ]:
j

In [ ]:
weight

In [ ]:
import numpy as np

In [ ]:
a = np.array([i, j, weight]).T

In [ ]:
from graph_tool.all import *

In [ ]:
openmp_set_num_threads(2)

In [ ]:
facts_norm.head()

In [ ]:
g = Graph(directed=False)
vp_fact = g.new_vertex_property('string')
for i in range(0, len(facts_norm)):
    v = g.add_vertex()
    vp_fact[v] = facts_norm['fact'][i]
g.vertex_properties['fact'] = vp_fact
ep_weight = g.new_edge_property('double')
g.add_edge_list(a, eprops=[ep_weight])
g.edge_properties['weight'] = ep_weight

In [ ]:
graph_draw(g, vertex_text=g.vp.fact, edge_pen_width=prop_to_size(g.ep.weight, mi=1, ma=5), output_size=(200, 200))

## 2.2. Social Network Science: Bipartite Graph of Work Concept Usage

In [ ]:
usages = pd.read_csv('data/sns/usages.txt', header='infer', delimiter='\t', encoding='utf-8')
usages.head()

In [ ]:
publications = pd.read_csv('data/sns/publications.txt', header='infer', delimiter='\t', encoding='utf-8')
publications.head()

In [ ]:
words = pd.read_csv('data/sns/words.txt', header='infer', delimiter='\t', encoding='utf-8')
words.head()

In [ ]:
foo = pd.merge(left=usages, right=publications[['publication_id', 'publication']], on='publication_id')
usages = pd.merge(left=foo, right=words, on='word_id')[['publication', 'word']]
usages.columns = ['transaction', 'fact']
usages['weight'] = 1
usages.head()

In [ ]:
len(usages)

### 2.2.1. Matrix Projection
#### 2.2.1.1. Publication Similarity Graph

In [ ]:
I_norm, transactions_norm = project_bipartite(df_selections=usages, projection='transactions', norm=True, remove_diagonal=True, sym=True)

In [ ]:
transactions_norm.head()

In [ ]:
I_norm.shape

In [ ]:
u = I_norm.nonzero()[0].tolist()
v = I_norm.nonzero()[1].tolist()
w = I_norm.data.tolist()
a = np.array([u, v, w]).T

In [ ]:
i = Graph(directed=False)
i_vp_transaction = i.new_vertex_property('string')
for Iter in range(0, len(transactions_norm)):
    v = i.add_vertex()
    i_vp_transaction[v] = transactions_norm['transaction'][Iter]
i.vertex_properties['transaction'] = i_vp_transaction
i_ep_similarity = i.new_edge_property('double')
i.add_edge_list(a, eprops=[i_ep_similarity])
i.edge_properties['similarity'] = i_ep_similarity

In [ ]:
i.num_vertices()

In [ ]:
i.num_edges()

In [ ]:
# the graph has too many edges to draw
# extract the minimum spanning tree
tree = min_spanning_tree(i, weights=i.ep.similarity)

In [ ]:
#get largest hub as root of the tree
k = i.get_out_degrees(vs=i.get_vertices())
max(k)

In [ ]:
ix = np.argmax(k)

In [ ]:
i.get_out_degrees(vs=[ix])

In [ ]:
v = i.vertex(ix)
print(i.vp.transaction[v])

In [ ]:
i_tree = GraphView(i, efilt=tree)

In [ ]:
i_tree.num_vertices()

In [ ]:
i_tree.num_edges()

In [ ]:
i_tree_pos = radial_tree_layout(i_tree, root=ix)

In [ ]:
graph_draw(i_tree, pos=i_tree_pos)

#### 2.2.1.2. Word Co-Usage Graph

In [ ]:
H_norm, facts_norm = project_bipartite(df_selections=usages, projection='facts', norm=True, remove_diagonal=True, sym=True)

In [ ]:
H_norm.shape

In [ ]:
x = H_norm.nonzero()[0].tolist()
y = H_norm.nonzero()[1].tolist()
z = H_norm.data.tolist()
b = np.array([x, y, z]).T

In [ ]:
h = Graph(directed=False)
h_vp_fact = h.new_vertex_property('string')
h_vp_weight = h.new_vertex_property('double')
for Iter in range(0, len(facts_norm)):
    v = h.add_vertex()
    h_vp_fact[v] = facts_norm['fact'][Iter]
    h_vp_weight[v] = facts_norm['weight'][Iter]
h.vertex_properties['fact'] = h_vp_fact
h.vertex_properties['weight'] = h_vp_weight
h_ep_weight = h.new_edge_property('double')
h.add_edge_list(b, eprops=[h_ep_weight])
h.edge_properties['weight'] = h_ep_weight

In [ ]:
h.num_vertices()

In [ ]:
h.num_edges()

In [ ]:
# the graph is to large to draw: filter the core
# get edge weight histogram
Min = min(h.ep.weight.a)
Max = max(h.ep.weight.a)
counts, bins = edge_hist(h, eprop=h.ep.weight, bins=np.logspace(np.log10(Min), np.log10(Max), 20))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(bins[-len(counts):], counts)
plt.xscale('log')
plt.yscale('log')
plt.title('unbehaved distribution of tie weights')
plt.xlabel('weight')
plt.ylabel('number of edges')

In [ ]:
# filter edges
minimum_edge_weight = 10
h_ep_core_edges = h.ep.weight.a > minimum_edge_weight

In [ ]:
h_efilt = GraphView(h, efilt=h_ep_core_edges)

In [ ]:
h_efilt.num_edges()

In [ ]:
h_efilt.num_vertices()

In [ ]:
# the graph still has all vertices
# just extract the largest component
lcc = label_largest_component(h_efilt)

In [ ]:
h_efilt_lcc = GraphView(h_efilt, vfilt=lcc)

In [ ]:
graph_draw(h_efilt_lcc, 
           #vertex_text=h_efilt_lcc.vp.fact, 
           #vertex_text_position=0, 
           vertex_size=prop_to_size(h_efilt_lcc.vp.weight, 1, 50), 
           edge_pen_width=prop_to_size(h_efilt_lcc.ep.weight, 1, 10))

In [ ]:
# or just show components with a minimum size
h_efilt_vp_component = h_efilt.new_vertex_property('int')
label_components(h_efilt, vprop=h_efilt_vp_component) # store id of component node belongs to in property map

In [ ]:
minimum_component_size = 2
c = h_efilt_vp_component.a.tolist()
core_components = list(set([x for x in c if c.count(x) >= minimum_component_size]))

In [ ]:
h_efilt_vp_core_component = h_efilt.new_vertex_property('bool')
for v in h_efilt.vertices():
    h_efilt_vp_core_component[v] = h_efilt_vp_component[v] in core_components

In [ ]:
h_efilt_vfilt = GraphView(h, vfilt=h_efilt_vp_core_component, efilt=h_ep_core_edges)

In [ ]:
h_efilt_vfilt.num_vertices()

In [ ]:
graph_draw(h_efilt_vfilt, 
           #vertex_text=h_efilt_vfilt.vp.fact, 
           #vertex_text_position=0, 
           vertex_size=prop_to_size(h_efilt_vfilt.vp.weight, 1, 50), 
           edge_pen_width=prop_to_size(h_efilt_vfilt.ep.weight, 1, 10))

## 2.3. Exercise
Filtering weak co-selection ties is a good way to extract the core when the tie strength distribution is very skewed - as is the case for the word co-usage network. When the tie strength distribution is not very skewed, it is better to use cohesion-based algorithms like <a href='http://intersci.ss.uci.edu/wiki/index.php/Cohesive_blocking'>k-component decomposition</a> to extract the core. The co-authorship network of Social Network Science is such a "well-behaved" graph:

In [ ]:
authorships = pd.read_csv('data/sns/authorships.txt', header='infer', delimiter='\t', encoding='utf-8')
# remove papers with exceptionally many authors
foo = authorships.groupby('publication_id').size().reset_index(name='num_authors')
foo = foo[foo['num_authors'] < (np.mean(foo['num_authors'])+3*np.std(foo['num_authors']))]['publication_id']
authorships = pd.merge(left=authorships, right=foo, on='publication_id')
authorships.head()

In [ ]:
publications.head()

In [ ]:
authors = pd.read_csv('data/sns/authors.txt', header='infer', delimiter='\t', encoding='utf-8')
authors.head()

In [ ]:
foo = pd.merge(left=authorships, right=publications[['publication_id', 'publication']], on='publication_id')
authorships = pd.merge(left=foo, right=authors, on='author_id')[['publication', 'author']]
authorships.columns = ['transaction', 'fact']
authorships['weight'] = 1
authorships.head()

k-component decomposition is not available in graph-tool, but an approximation is: Every k-component is also a k-core but not vice versa. Use <a href=''>kcore_decomposition</a> to extract the core of the co-authorship network of authors that are connected if they write papers together.